In [1]:
from datasets import load_dataset

In [2]:
#dataset = load_dataset('json',data_files=['train.json','validation.json'])
dataset = load_dataset('json',data_files={'train':'../Datasets/train_Bisnode_Ocdata_Full.json','test':'../Datasets/test_bisnode_ocdata_FULL.json'})

Using custom data configuration default-6a8ddd758d7e654e
Reusing dataset json (C:\Users\liuyw\.cache\huggingface\datasets\json\default-6a8ddd758d7e654e\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'labels', 'sentence1', 'sentence2'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['idx', 'labels', 'sentence1', 'sentence2'],
        num_rows: 500
    })
})

In [4]:
dataset_fist_six = load_dataset('json',data_files={'input_first_six':'../Datasets/input_bisnode_ocdata_first_six_Full.json'})

Using custom data configuration default-66b403716b42f9a5
Reusing dataset json (C:\Users\liuyw\.cache\huggingface\datasets\json\default-66b403716b42f9a5\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset_fist_six

DatasetDict({
    input_first_six: Dataset({
        features: ['idx', 'sentence1', 'sentence2'],
        num_rows: 120000
    })
})

In [6]:
dataset["test"][0:10]

{'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'labels': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'sentence1': ['COL name VAL Holz-Huber Inh. Albert Huber e.K. COL address VAL Abensberger Str. 39-41, 84048 Kelheim COL number VAL Amtsgericht Regensburg HRA 5944 COL state VAL Bayern',
  'COL name VAL Elektro Felger GmbH COL address VAL Roentgenstr. 30, 71229 Boeblingen COL number VAL Amtsgericht Stuttgart HRB 251812 COL state VAL Baden-Wuerttemberg',
  'COL name VAL Bickebach GmbH, Schlosserei, Stahl- und Metallbau COL address VAL Wernher-von-Braun-Str. 18, 86368 Lk.Augsburg COL number VAL Amtsgericht Augsburg HRB 17112 COL state VAL Bayern',
  'COL name VAL G+N Elektrotechnik GmbH COL address VAL Nollenpad 43, 45894 Gelsenkirchen COL number VAL Amtsgericht Gelsenkirchen HRB 7032 COL state VAL Nordrhein-Westfalen',
  'COL name VAL Humboldt- und Agricola-Buchhandlung e.K. COL address VAL Moritzstr. 20, 09111 Chemnitz COL number VAL Amtsgericht Chemnitz HRA 3843 COL state VAL Sachsen',
  'COL name VAL M

In [7]:
from transformers import AutoTokenizer

In [8]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
def tokenize_function(example):
    return tokenizer(example["sentence1"],example["sentence2"],padding="max_length",truncation=True,max_length=128)

In [10]:
tokenized_datasets = dataset.map(tokenize_function,batched=True)
print(tokenized_datasets.column_names)

Loading cached processed dataset at C:\Users\liuyw\.cache\huggingface\datasets\json\default-6a8ddd758d7e654e\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50\cache-965385d1dad5bf54.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids'], 'test': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids']}


In [11]:
tokenized_datasets_first_six = dataset_fist_six.map(tokenize_function,batched=True)
print(dataset_fist_six.column_names)

Loading cached processed dataset at C:\Users\liuyw\.cache\huggingface\datasets\json\default-66b403716b42f9a5\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50\cache-f219372931e27efe.arrow


{'input_first_six': ['idx', 'sentence1', 'sentence2']}


In [12]:
from transformers import DataCollatorWithPadding

In [13]:
data_collator = DataCollatorWithPadding(tokenizer)

In [14]:
from transformers import BertForSequenceClassification

In [15]:
model = BertForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [16]:
from transformers import TrainingArguments

In [17]:
training_args = TrainingArguments("test-trainer")

In [18]:
from transformers import Trainer

trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json
Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test-trainer\checkpoint-1000
Configuration saved in test-trainer\checkpoint-1000\config.json
Model weights saved in test-trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.08299636289808485, metrics={'train_runtime': 2768.9599, 'train_samples_per_second': 3.25, 'train_steps_per_second': 0.406, 'total_flos': 591999874560000.0, 'train_loss': 0.08299636289808485, 'epoch': 3.0})

In [19]:
predictions = trainer.predict(tokenized_datasets['test'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [24]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds = np.argmax(predictions.predictions,axis=1)

metric.compute(predictions=preds,references=predictions.label_ids)

{'f1': 0.8994974874371858}

In [25]:
metric_p = load_metric("precision")

preds = np.argmax(predictions.predictions,axis=1)

metric_p.compute(predictions=preds,references=predictions.label_ids)

{'precision': 0.8861386138613861}

In [26]:
metric_r = load_metric("recall")

preds = np.argmax(predictions.predictions,axis=1)

metric_r.compute(predictions=preds,references=predictions.label_ids)

{'recall': 0.9132653061224489}

In [28]:
predictions_first_six = trainer.predict(tokenized_datasets_first_six['input_first_six'])


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Prediction *****
  Num examples = 120000
  Batch size = 8


In [29]:
type(predictions_first_six)

transformers.trainer_utils.PredictionOutput

In [30]:
predictions_first_six.predictions.shape

(120000, 2)

In [31]:
preds = np.argmax(predictions_first_six.predictions,axis=1)
len(preds)

120000

In [32]:
with open('../Datasets/final_Bert_bisnode_ocdata_Full.txt', 'w') as f:
    for item in preds:
        f.write("%s\n" % item)

In [33]:
dataset_second_six = load_dataset('json',data_files={'input_second_six':'../Datasets/input_bisnode_ocdata_second_six_FULL.json'})

Using custom data configuration default-70bccf66e5fda317


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:\Users\liuyw\.cache\huggingface\datasets\json\default-70bccf66e5fda317\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
tokenized_datasets_second_six = dataset_second_six.map(tokenize_function,batched=True)
print(dataset_second_six.column_names)

  0%|          | 0/126 [00:00<?, ?ba/s]

{'input_second_six': ['idx', 'sentence1', 'sentence2']}


In [35]:
predictions_second_six = trainer.predict(tokenized_datasets_second_six['input_second_six'])


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Prediction *****
  Num examples = 125808
  Batch size = 8


In [36]:
predictions_second_six.predictions.shape

(125808, 2)

In [37]:
preds_second = np.argmax(predictions_second_six.predictions,axis=1)
len(preds_second)
preds_second

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [38]:
with open('../Datasets/final_Bert_bisnode_ocdata_second_FULL.txt', 'w') as f:
    for item in preds_second:
        f.write("%s\n" % item)